# Avanessova, 2025 Dissertation WOMBAT Results Replication

National Renewable Energy Laboratory\
Rob Hammond\
Last Updated: 13 October 2025

**Note**: The results in this notebook will differ from the published results because this example
was created during the v0.12 release cycle, and the published results are were using WOMBAT
v0.8.1. As such there are numerous bug fixes, improvements, and new features in the model that
will change the results.

For complete details on this study, please see: http://dx.doi.org/10.7488/era/5854.

In [1]:
from time import perf_counter
from pathlib import Path
from itertools import product

import numpy as np
import pandas as pd

from wombat import Simulation
from wombat.core.post_processor import _check_frequency, Metrics
from wombat.core.library import AVANESSOVA_DISS

pd.options.display.float_format = "{:,.3f}".format

In [2]:
# NOTE: the original analysis ran 50 simulations, but this will
# run 5 each for demonstration purposes.
N = 5

ANALYSIS_LIBRARY = AVANESSOVA_DISS
RESULTS_DIR = AVANESSOVA_DISS / "results"

RESULTS_BASE = {
    "opex_total": 0,
    "opex_levelized": 0,
    "vessel_annual": 0,
    "vessel_levelized": 0,
    "ctv_annual": 0,
    "rov_annual": 0,
    "cab_annual": 0,
    "ahv_annual": 0,
    "jackup_annual": 0,
    "tugboat_annual": 0,
    "cab_mobilization": 0,
    "ahv_mobilization": 0,
    "jackup_mobilization": 0,
    "activities_annual": 0,
    "labor_annual": 0,
    "activities_levelized": 0,
    "labor_levelized": 0,
    "energy": 0,
    "opex_energy": 0,
    "time_availability": 0,
    "energy_availability": 0,
    "planned_visits": 0,
    "unplanned_visits": 0,
    "time_to_repair_start": 0,
    "quayside_annual": 0,
    "gcf": 0,
    "ncf": 0,
}
results_columns = [*RESULTS_BASE]

results_order = [
    "opex_total",
    "opex_levelized",
    "vessel_annual",
    "vessel_levelized",
    "quayside_annual",
    "ctv_annual",
    "rov_annual",
    "cab_annual",
    "ahv_annual",
    "jackup_annual",
    "tugboat_annual",
    "cab_mobilization",
    "ahv_mobilization",
    "jackup_mobilization",
    "activities_annual",
    "labor_annual",
    "activities_levelized",
    "labor_levelized",
    "energy",
    "opex_energy",
    "time_availability",
    "energy_availability",
    "planned_visits",
    "unplanned_visits",
    "time_to_repair_start",
    "gcf",
    "ncf",
]
vessel_results_order = [
    "Weather Delay",
    "Vessel Usage",
    "Mobilization",
    "Vessel Inactive",
    "Vessel Cost",
]

## Results Gathering Functionality

In [3]:
def equipment_breakdowns(
    metrics: Metrics, frequency: str, by_category: bool = False
) -> pd.DataFrame:
    """Calculates the producitivty cost breakdowns for the simulation at a project,
    annual, or monthly level that can be broken out to include the equipment and
    labor components.

    .. note:: Doesn't produce a value if there's no cost associated with a "reason".

    Parameters
    ----------
    frequency : str
        One of "project", "annual", "monthly", or "month-year".
    by_category : bool, optional
        Indicates whether to include the equipment and labor categories (True) or
        not (False), by default False.

    Returns
    -------
    pd.DataFrame
        Returns pandas ``DataFrame`` with columns:
            - year (if appropriate for frequency)
            - month (if appropriate for frequency)
            - reason
            - time (hours)
            - equipment_cost (if by_category == ``True``)

    Raises
    ------
    ValueError
        If ``frequency`` is not one of "project", "annual", "monthly", or
        "month-year".
    ValueError
        If ``by_category`` is not one of ``True`` or ``False``.
    """
    frequency = _check_frequency(frequency, which="all")
    if not isinstance(by_category, bool):
        raise ValueError("``by_equipment`` must be one of ``True`` or ``False``")

    group_filter = ["action", "reason", "additional", "agent"]
    if frequency in ("annual", "month-year"):
        group_filter.insert(0, "year")
    elif frequency == "monthly":
        group_filter.insert(0, "month")
    if frequency == "month-year":
        group_filter.insert(1, "month")

    action_list = [
        "delay",
        "repair",
        "maintenance",
        "mobilization",
        "transferring crew",
        "traveling",
        "towing",
        "mooring reconnection",
        "unmooring",
    ]
    equipment = metrics.events[
        metrics.events[metrics._equipment_cost] > 0
    ].agent.unique()
    time_costs = (
        metrics.events.loc[
            metrics.events.agent.isin(equipment)
            & metrics.events.action.isin(action_list)
            & ~metrics.events.additional.isin(["work is complete"]),
            group_filter + [metrics._equipment_cost, "duration"],
        ]
        .groupby(group_filter)
        .sum()
        .reset_index()
    )
    time_costs["display_reason"] = [""] * time_costs.shape[0]

    non_shift_hours = (
        "not in working hours",
        "work shift has ended; waiting for next shift to start",
        "no more return visits will be made",
        "will return next year",
        "waiting for next operational period",
    )
    weather_hours = ("weather delay", "weather unsuitable to transfer crew")
    time_costs.loc[
        (time_costs.action == "delay") & (time_costs.additional.isin(non_shift_hours)),
        "display_reason",
    ] = "Not in Shift"
    time_costs.loc[time_costs.action == "repair", "display_reason"] = "Repair"
    time_costs.loc[time_costs.action == "maintenance", "display_reason"] = "Maintenance"
    time_costs.loc[time_costs.action == "transferring crew", "display_reason"] = (
        "Crew Transfer"
    )
    time_costs.loc[time_costs.action == "traveling", "display_reason"] = "Site Travel"
    time_costs.loc[time_costs.action == "mobilization", "display_reason"] = (
        "Mobilization"
    )
    time_costs.loc[time_costs.additional.isin(weather_hours), "display_reason"] = (
        "Weather Delay"
    )
    time_costs.loc[time_costs.reason == "no requests", "display_reason"] = "No Requests"

    time_costs.reason = time_costs.display_reason

    drop_columns = ["display_reason", "additional", "action"]
    group_filter.pop(group_filter.index("additional"))
    group_filter.pop(group_filter.index("action"))
    time_costs = time_costs.drop(columns=drop_columns)
    time_costs = time_costs.groupby(group_filter).sum().reset_index()

    month_year = frequency == "month-year"
    if frequency in ("annual", "month-year"):
        years = time_costs.year.unique()
        reasons = time_costs.reason.unique()
        comparison_values = product(years, reasons)
        if month_year:
            months = time_costs.month.unique()
            comparison_values = product(years, months, reasons)

        zeros = np.zeros(time_costs.shape[1] - 2).tolist()
        for _year, *_month, _reason in comparison_values:
            row_filter = time_costs.year.values == _year
            row = [_year, _reason] + zeros
            if month_year:
                _month = _month[0]
                row_filter &= time_costs.month.values == _month
                row = [_year, _month, _reason] + zeros[:-1]

            row_filter &= time_costs.reason.values == _reason
            if time_costs.loc[row_filter].size > 0:
                continue
            time_costs.loc[time_costs.shape[0]] = row
    elif frequency == "monthly":
        months = time_costs.month.unique()
        reasons = time_costs.reason.unique()
        comparison_values = product(months, reasons)
        zeros = np.zeros(time_costs.shape[1] - 2).tolist()
        for _month, _reason in comparison_values:
            row_filter = time_costs.month.values == _month
            row_filter &= time_costs.reason.values == _reason
            row = [_month, _reason] + zeros
            if time_costs.loc[row_filter].size > 0:
                continue
            time_costs.loc[time_costs.shape[0]] = row

    new_sort = [
        "Maintenance",
        "Repair",
        "Crew Transfer",
        "Site Travel",
        "Mobilization",
        "Weather Delay",
        "No Requests",
        "Not in Shift",
    ]
    time_costs.reason = pd.Categorical(time_costs.reason, new_sort)
    time_costs = time_costs.set_index(group_filter)
    if frequency == "project":
        return time_costs.sort_values(by="reason")
    if frequency == "annual":
        return time_costs.sort_values(by=["year", "reason"])
    if frequency == "monthly":
        return time_costs.sort_values(by=["month", "reason"])
    return time_costs.sort_values(by=["year", "month", "reason"])


def vessel_summary(metrics: Metrics, frequency="project") -> pd.DataFrame:
    keep = ["Weather Delay", "Vessel Usage", "Mobilization", "Vessel Inactive"]
    order = [
        "Crew Transfer Vessel 1",
        "Crew Transfer Vessel 2",
        "Crew Transfer Vessel 3",
        "ROV Support Vessel",
        "Cable Laying Vessel",
        "Anchor Handling Vessel",
        "Heavy Lift Vessel",
        "Tugboat 1",
    ]
    vessel_results = metrics.equipment_labor_cost_breakdowns(
        frequency, by_category=True, by_equipment=True
    )
    vessel_results = vessel_results.unstack(level="equipment_name").fillna(0)

    if "Towing" not in vessel_results.index:
        vessel_results.loc["Towing"] = 0

    vessel_results.loc["Vessel Usage"] = 0
    vessel_results.loc["Vessel Inactive"] = 0
    vessel_results.loc["Vessel Usage"] = (
        vessel_results.loc[
            ["Maintenance", "Repair", "Crew Transfer", "Site Travel", "Towing"], :
        ]
        .sum(axis=0)
        .to_frame(name="Vessel Usage")
        .T.values
    )
    vessel_results.loc["Vessel Inactive"] = (
        vessel_results.loc[["Not in Shift", "No Requests"], :]
        .sum(axis=0)
        .to_frame(name="Vessel Inactive")
        .T.values
    )
    vessel_results = vessel_results.loc[keep]

    vessel_results = pd.concat(
        [
            vessel_results.loc[:, "total_hours"],
            vessel_results.loc[:, "equipment_cost"]
            .sum()
            .to_frame(name="Vessel Cost")
            .T,
        ]
    )
    vessel_results.columns.name = None
    vessel_results.index.name = "Name"
    return vessel_results[[el for el in order if el in vessel_results.columns]]


def gather_results(sim: Simulation) -> dict:
    """Calculates the primary set of results without granular breakdowns."""
    project_years = 20
    n_turbines = len(sim.windfarm.turbine_id)
    metrics = sim.metrics
    events = metrics.events
    capacity_kw = sim.windfarm.capacity
    results = {}

    opex = metrics.opex("project").values[0][0]
    opex_levelized = opex / capacity_kw / project_years

    port_annual = metrics.port_fees("project").values[0][0] / project_years

    mobilization = (
        events.loc[events.action == "mobilization", ["agent", "total_cost"]]
        .groupby("agent")
        .sum()
    )
    cab_mobilization = (
        mobilization.loc[
            [el for el in mobilization.index if "Cable Laying Vessel" in el]
        ].values.sum()
        / project_years
    )
    ahv_mobilization = (
        mobilization.loc[
            [el for el in mobilization.index if "Anchor Handling Vessel" in el]
        ].values.sum()
        / project_years
    )
    jackup_mobilization = (
        mobilization.loc[
            [el for el in mobilization.index if "Heavy Lift Vessel" in el]
        ].values.sum()
        / project_years
    )

    vessels = metrics.equipment_costs(frequency="project", by_equipment=True)
    vessel_annual = vessels.values.sum() / project_years
    vessel_levelized = vessel_annual / capacity_kw
    ctv_annual = (
        vessels[
            [el for el in vessels.columns if "Crew Transfer Vessel" in el]
        ].values.sum()
        / project_years
    )
    rov_annual = (
        vessels[
            [el for el in vessels.columns if "ROV Support Vessel" in el]
        ].values.sum()
        / project_years
    )
    cab_annual = (
        vessels[
            [el for el in vessels.columns if "Cable Laying Vessel" in el]
        ].values.sum()
        / project_years
        - cab_mobilization
    )
    ahv_annual = (
        vessels[
            [el for el in vessels.columns if "Anchor Handling Vessel" in el]
        ].values.sum()
        / project_years
        - ahv_mobilization
    )
    jackup_annual = (
        vessels[
            [el for el in vessels.columns if "Heavy Lift Vessel" in el]
        ].values.sum()
        / project_years
        - jackup_mobilization
    )
    tugboat_annual = (
        vessels[[el for el in vessels.columns if "Tugboat" in el]].values.sum()
        / project_years
    )

    materials_cost = metrics.events.materials_cost.sum()
    materials_annual = materials_cost / project_years
    materials_levelized = materials_annual / capacity_kw

    labor_annual = (
        metrics.project_fixed_costs("project", "high").labor.values[0]
        + metrics.labor_costs("project", False).total_labor_cost.values[0]
    ) / project_years
    labor_levelized = labor_annual / capacity_kw

    energy = metrics.power_production("project", "windfarm", "kwh").values[0][0]
    opex_energy = (
        opex / metrics.power_production("project", "windfarm", "mwh").values[0][0]
    )
    time_availability = metrics.time_based_availability(
        frequency="project", by="windfarm"
    ).values[0][0]
    energy_availability = metrics.production_based_availability(
        frequency="project", by="windfarm"
    ).values[0][0]

    serviced_requests = pd.Series(
        events.loc[
            events.agent.isin(metrics.service_equipment_names)
        ].request_id.unique(),
        name="request_id",
    )
    n_maintenance = (
        serviced_requests.str.startswith("MNT").sum() / n_turbines / project_years
    )
    n_repair = (
        serviced_requests.str.startswith("RPR").sum() / n_turbines / project_years
    )

    timing = metrics.process_times()
    average_time_to_start = timing.sum().T.time_to_start / timing.sum().T.N

    gcf = metrics.capacity_factor("gross", "project", "windfarm").values[0][0]
    ncf = metrics.capacity_factor("net", "project", "windfarm").values[0][0]

    results["opex_total"] = opex
    results["opex_levelized"] = opex_levelized
    results["vessel_annual"] = vessel_annual
    results["vessel_levelized"] = vessel_levelized
    results["quayside_annual"] = port_annual
    results["ctv_annual"] = ctv_annual
    results["rov_annual"] = rov_annual
    results["cab_annual"] = cab_annual
    results["ahv_annual"] = ahv_annual
    results["jackup_annual"] = jackup_annual
    results["tugboat_annual"] = tugboat_annual
    results["cab_mobilization"] = cab_mobilization
    results["ahv_mobilization"] = ahv_mobilization
    results["jackup_mobilization"] = jackup_mobilization
    results["activities_annual"] = materials_annual
    results["labor_annual"] = labor_annual
    results["activities_levelized"] = materials_levelized
    results["labor_levelized"] = labor_levelized
    results["energy"] = energy
    results["opex_energy"] = opex_energy
    results["time_availability"] = time_availability
    results["energy_availability"] = energy_availability
    results["planned_visits"] = n_maintenance
    results["unplanned_visits"] = n_repair
    results["time_to_repair_start"] = average_time_to_start
    results["gcf"] = gcf
    results["ncf"] = ncf

    return results

## Analysis Running Functionality

In [4]:
def load_and_run(name: str, i: int, rng) -> Simulation:
    """Run a given configuration, and output the summary runtime stats and primary results."""
    print(f"{name.rjust(40)} | {i:>3.0f}", end=" | ")
    start = perf_counter()
    sim = Simulation(ANALYSIS_LIBRARY, f"{name}.yaml", random_generator=rng)
    end = perf_counter()
    print(f"Load: {(end - start) / 60:.2f} m", end=" | ")

    start = perf_counter()
    sim.run()
    end = perf_counter()
    print(f"Run: {(end - start) / 60:5.2f} m", end=" | ")

    avail = sim.metrics.production_based_availability(
        frequency="project", by="windfarm"
    ).values[0][0]
    opex = sim.metrics.opex("project").values[0][0] / sim.windfarm.capacity
    print(f"Avail: {avail:6.2%} | Opex ($/kw): {opex: 8,.2f}")
    sim.env.cleanup_log_files()
    return sim


def compile_results(sim: Simulation) -> dict:
    """Gathers the primary results, vessel usage summaries, and activities summaries."""
    results = gather_results(sim)

    # Calculate the vessel breakdown
    vessel_results = vessel_summary(sim.metrics)

    # Summarize the repair activities
    materials = sim.metrics.events.loc[
        sim.metrics.events.materials_cost > 0, ["part_name", "reason", "materials_cost"]
    ]
    materials.loc[materials.part_name.str.startswith("CB"), "part_name"] = "cable"
    materials_results = (
        materials.groupby(["part_name", "reason"])
        .agg(["count", "sum"])
        .sort_index()
        .T.droplevel(0)
        .T
    )

    return results, vessel_results, materials_results

In [5]:
configs = [
    "layout_1_in_situ_base",
    "layout_1_in_situ_base_24hr",
    "layout_1_in_situ_no_major",
    "layout_1_in_situ_increase_maintenance",
    "layout_1_in_situ_increase_failure",
    "layout_1_in_situ_increase_failure_24hr",
    "layout_1_in_situ_increase_wave",
    "layout_1_tow_base",
    "layout_1_tow_base_24hr",
    "layout_1_tow_increase_wave",
    "layout_1_tow_increase_failure",
    "layout_1_tow_increase_maintenance",
    "layout_2_tow_base",
    "layout_2_in_situ_base",
]

results_dict = {
    "primary": {},
    "vessel": {},
    "activities": {},
}
for config in configs:
    results_list = []
    vessel_results_list = []
    activities_results_list = []

    # Reset the random generator for the scenario, and run each scenario 50 times
    rng = np.random.default_rng(seed=34)
    for i in range(N):
        sim = load_and_run(config, i, rng)
        results, vessel_results, materials_results = compile_results(sim)

        # Create the primary results data frame and append each to the list of results
        results = pd.DataFrame.from_dict(results, orient="index", columns=[config]).loc[
            results_order
        ]
        results.index.name = "metric"
        results_list.append(results)
        vessel_results_list.append(vessel_results)
        activities_results_list.append(materials_results)

        sim.env.cleanup_log_files()

    # Concatenate each set of results into a single dataframe
    results = (
        pd.concat(results_list)
        .groupby(["metric"])
        .agg(["mean", "std"])
        .loc[results_order]
    )
    vessel_results = (
        pd.concat(vessel_results_list)
        .groupby(["Name"])
        .agg(["mean", "std"])
        .loc[vessel_results_order]
    )
    activities_results = (
        pd.concat(activities_results_list)
        .groupby(["part_name", "reason"])
        .agg(["mean", "std"])
    )

    # Set the main attributes to make the saved results legible
    vessel_results.columns.names = ["Vessel", "Value"]
    vessel_results = vessel_results.unstack().to_frame()
    vessel_results.index = (
        vessel_results.index.swaplevel("Vessel", "Name")
        .swaplevel("Value", "Vessel")
        .set_names(["Metric", "Vessel", "Value"])
    )
    vessel_results = vessel_results.unstack()
    vessel_results.columns = vessel_results.columns.droplevel(0).set_names([None])

    # Save to file
    results_dict["primary"][config] = results
    results_dict["vessel"][config] = vessel_results
    results_dict["activities"][config] = activities_results

                   layout_1_in_situ_base |   0 | Load: 0.01 m | Run:  0.81 m | Avail: 93.05% | Opex ($/kw):  1,132.75
                   layout_1_in_situ_base |   1 | Load: 0.01 m | Run:  0.81 m | Avail: 93.12% | Opex ($/kw):  1,181.78
                   layout_1_in_situ_base |   2 | Load: 0.01 m | Run:  0.84 m | Avail: 93.12% | Opex ($/kw):  1,111.74
                   layout_1_in_situ_base |   3 | Load: 0.01 m | Run:  0.83 m | Avail: 93.17% | Opex ($/kw):  1,134.37
                   layout_1_in_situ_base |   4 | Load: 0.01 m | Run:  0.85 m | Avail: 93.12% | Opex ($/kw):  1,150.53
              layout_1_in_situ_base_24hr |   0 | Load: 0.01 m | Run:  0.69 m | Avail: 95.16% | Opex ($/kw):  1,087.79
              layout_1_in_situ_base_24hr |   1 | Load: 0.01 m | Run:  0.72 m | Avail: 95.15% | Opex ($/kw):  1,107.02
              layout_1_in_situ_base_24hr |   2 | Load: 0.01 m | Run:  0.75 m | Avail: 95.12% | Opex ($/kw):  1,096.10
              layout_1_in_situ_base_24hr |   3 | Load: 0

## Results Gathering


In [6]:
column_order = [
    "layout_1_in_situ_base",
    "layout_1_in_situ_no_major",
    "layout_1_in_situ_increase_failure",
    "layout_1_in_situ_increase_maintenance",
    "layout_1_in_situ_increase_wave",
    "layout_1_tow_base",
    "layout_1_tow_increase_failure",
    "layout_1_tow_increase_maintenance",
    "layout_1_tow_increase_wave",
    "layout_2_tow_base",
    "layout_2_in_situ_base",
    "layout_1_in_situ_base_24hr",
    "layout_1_in_situ_increase_failure_24hr",
    "layout_1_tow_base_24hr",
]


def calculate_summary_stats(name, df):
    df = df.droplevel(0, axis=1)
    mean = df[["mean"]].rename(columns={"mean": name})
    std = df[["std"]].rename(columns={"std": name})
    return mean, std


def calculate_vessel_stats(name, df):
    return df[["mean"]].rename(columns={"mean": name}), df[["std"]].rename(
        columns={"std": name}
    )


def calculate_activities_stats(name, df):
    ix_drop = ~df.index.get_level_values("part_name").isna()
    count_mean = (
        df[["count"]].droplevel(0, axis=1)[["mean"]].rename(columns={"mean": name})
    )
    count_std = (
        df[["count"]].droplevel(0, axis=1)[["std"]].rename(columns={"std": name})
    )
    sum_mean = df[["sum"]].droplevel(0, axis=1)[["mean"]].rename(columns={"mean": name})
    sum_std = df[["sum"]].droplevel(0, axis=1)[["std"]].rename(columns={"std": name})
    return count_mean, count_std, sum_mean, sum_std


summary_list = [
    el
    for el in RESULTS_DIR.iterdir()
    if el.name.startswith("layout")
    and el.name.endswith("summary.csv")
    and "vessel" not in el.name
]
vessel_list = [
    el
    for el in RESULTS_DIR.iterdir()
    if el.name.startswith("layout")
    and el.name.endswith("summary.csv")
    and "vessel" in el.name
]
activities_list = [
    el
    for el in RESULTS_DIR.iterdir()
    if el.name.startswith("layout") and el.name.endswith("activities.csv")
]

# Split between mean and standard deviation data

# Overall results
summary_data = [calculate_summary_stats(*el) for el in results_dict["primary"].items()]
summary_means = pd.concat([el[0] for el in summary_data], axis=1, join="outer").fillna(
    0.0
)
summary_stds = pd.concat([el[1] for el in summary_data], axis=1, join="outer").fillna(
    0.0
)

# Vessels
vessel_data = [calculate_vessel_stats(*el) for el in results_dict["vessel"].items()]
vessel_means = (
    pd.concat([el[0] for el in vessel_data], axis=1, join="outer").fillna(0.0) / 20.0
)
vessel_stds = (
    pd.concat([el[1] for el in vessel_data], axis=1, join="outer").fillna(0.0) / 20.0
)

vessel_means = vessel_means.sort_index().reset_index(drop=False)
vessel_means.Metric = pd.Categorical(
    vessel_means["Metric"],
    ordered=True,
    categories=[
        "Weather Delay",
        "Mobilization",
        "Vessel Usage",
        "Vessel Inactive",
        "Vessel Cost",
    ],
)
vessel_means.Vessel = pd.Categorical(
    vessel_means["Vessel"],
    ordered=True,
    categories=[
        "Crew Transfer Vessel - 1",
        "Crew Transfer Vessel - 2",
        "Crew Transfer Vessel - 3",
        "ROV Support Vessel",
        "Cable Laying Vessel",
        "Anchor Handling Vessel",
        "Heavy Lift Vessel",
        "Tugboat 1",
    ],
)
vessel_means = vessel_means.sort_values(["Metric", "Vessel"]).set_index(
    ["Metric", "Vessel"]
)

vessel_stds = vessel_stds.sort_index().reset_index(drop=False)
vessel_stds.Metric = pd.Categorical(
    vessel_stds["Metric"],
    ordered=True,
    categories=[
        "Weather Delay",
        "Mobilization",
        "Vessel Usage",
        "Vessel Inactive",
        "Vessel Cost",
    ],
)
vessel_stds.Vessel = pd.Categorical(
    vessel_stds["Vessel"],
    ordered=True,
    categories=[
        "Crew Transfer Vessel - 1",
        "Crew Transfer Vessel - 2",
        "Crew Transfer Vessel - 3",
        "ROV Support Vessel",
        "Cable Laying Vessel",
        "Anchor Handling Vessel",
        "Heavy Lift Vessel",
        "Tugboat 1",
    ],
)
vessel_stds = vessel_stds.sort_values(["Metric", "Vessel"]).set_index(
    ["Metric", "Vessel"]
)

# Activities summary
activities_data = [
    calculate_activities_stats(*el) for el in results_dict["activities"].items()
]

count_means = pd.concat([el[0] for el in activities_data], axis=1, join="outer").fillna(
    0.0
)[column_order]
count_stds = pd.concat([el[1] for el in activities_data], axis=1, join="outer").fillna(
    0.0
)[column_order]
sum_means = pd.concat([el[2] for el in activities_data], axis=1, join="outer").fillna(
    0.0
)[column_order]
sum_stds = pd.concat([el[3] for el in activities_data], axis=1, join="outer").fillna(
    0.0
)[column_order]

summary_means = summary_means[column_order]
summary_stds = summary_stds[column_order]
vessel_means = vessel_means[column_order]
vessel_stds = vessel_stds[column_order]

In [7]:
summary_means

,layout_1_in_situ_base,layout_1_in_situ_no_major,layout_1_in_situ_increase_failure,layout_1_in_situ_increase_maintenance,layout_1_in_situ_increase_wave,layout_1_tow_base,layout_1_tow_increase_failure,layout_1_tow_increase_maintenance,layout_1_tow_increase_wave,layout_2_tow_base,layout_2_in_situ_base,layout_1_in_situ_base_24hr,layout_1_in_situ_increase_failure_24hr,layout_1_tow_base_24hr
metric,,,,,,,,,,,,,,
opex_total,"685,338,709.875","502,849,645.565","1,161,292,754.234","687,391,585.545","660,941,569.828","808,807,735.185","1,211,366,611.759","815,345,471.900","801,191,529.129","807,675,492.692","698,943,702.799","658,969,586.799","1,120,561,355.173","812,422,182.539"
opex_levelized,57.112,41.904,96.774,57.283,55.078,67.401,100.947,67.945,66.766,67.306,58.245,54.914,93.380,67.702
vessel_annual,"11,879,520.021","4,416,686.885","18,325,853.395","12,195,069.508","11,348,914.548","6,310,451.347","7,689,981.289","6,617,803.923","6,675,270.513","6,225,043.406","12,512,395.625","9,966,689.379","14,081,114.691","5,724,130.301"
vessel_levelized,19.799,7.361,30.543,20.325,18.915,10.517,12.817,11.030,11.125,10.375,20.854,16.611,23.469,9.540
quayside_annual,0.000,0.000,0.000,0.000,0.000,"11,862,500.000","11,862,500.000","11,862,500.000","11,862,500.000","11,862,500.000",0.000,0.000,0.000,"11,862,500.000"
ctv_annual,"2,738,881.691","2,738,881.466","2,738,895.504","2,738,893.943","2,738,832.921","2,738,881.520","2,738,892.602","2,738,893.666","2,738,833.621","2,738,880.997","2,738,881.811","2,738,820.349","2,738,783.583","2,738,822.817"
rov_annual,"1,825,736.499","1,677,805.419","1,890,200.032","2,173,903.567","1,734,259.712","1,823,791.299","1,970,584.588","2,185,506.323","1,775,809.688","1,799,714.515","1,788,680.426","1,358,006.046","1,354,359.559","1,399,965.840"
cab_annual,"595,193.256",0.000,"920,045.356","429,981.130","1,345,091.692","532,675.157","833,017.323","577,781.755","1,276,340.613","482,248.133","559,201.186","396,454.472","728,258.937","432,742.329"
ahv_annual,"45,753.978",0.000,"22,066.310","31,114.519","41,760.449","82,355.913","79,353.201","58,667.309","38,047.320","48,042.128","33,371.430","43,677.146","31,144.568","40,118.353"


In [8]:
vessel_means

layout_1_in_situ_base  \
Metric          Vessel                                          
Weather Delay   ROV Support Vessel                     45.909   
                Cable Laying Vessel                    23.377   
                Anchor Handling Vessel                  2.128   
                Heavy Lift Vessel                      91.514   
                Tugboat 1                               0.000   
Mobilization    ROV Support Vessel                      0.000   
                Cable Laying Vessel                   134.400   
                Anchor Handling Vessel                 21.840   
                Heavy Lift Vessel                     672.000   
                Tugboat 1                               0.000   
Vessel Usage    ROV Support Vessel                    803.004   
                Cable Laying Vessel                    66.625   
                Anchor Handling Vessel                 11.626   
                Heavy Lift Vessel                     277.527   
                Tugboat 1                               0.000   
Vessel Inactive ROV Support Vessel                  4,504.480   
                Cable Laying Vessel                    88.556   
                Anchor Handling Vessel                 12.706   
                Heavy Lift Vessel                     270.163   
                Tugboat 1                               0.000   
Vessel Cost     ROV Support Vessel              1,825,736.499   
                Cable Laying Vessel               990,193.256   
                Anchor Handling Vessel             88,003.978   
                Heavy Lift Vessel               6,236,704.597   
                Tugboat 1                               0.000   

                                        layout_1_in_situ_no_major  \
Metric          Vessel                                              
Weather Delay   ROV Support Vessel                         44.500   
                Cable Laying Vessel                         0.000   
                Anchor Handling Vessel                      0.000   
                Heavy Lift Vessel                           0.000   
                Tugboat 1                                   0.000   
Mobilization    ROV Support Vessel                          0.000   
                Cable Laying Vessel                         0.000   
                Anchor Handling Vessel                      0.000   
                Heavy Lift Vessel                           0.000   
                Tugboat 1                                   0.000   
Vessel Usage    ROV Support Vessel                        804.883   
                Cable Laying Vessel                         0.000   
                Anchor Handling Vessel                      0.000   
                Heavy Lift Vessel                           0.000   
                Tugboat 1                                   0.000   
Vessel Inactive ROV Support Vessel                      3,944.958   
                Cable Laying Vessel                         0.000   
                Anchor Handling Vessel                      0.000   
                Heavy Lift Vessel                           0.000   
                Tugboat 1                                   0.000   
Vessel Cost     ROV Support Vessel                  1,677,805.419   
                Cable Laying Vessel                         0.000   
                Anchor Handling Vessel                      0.000   
                Heavy Lift Vessel                           0.000   
                Tugboat 1                                   0.000   

                                        layout_1_in_situ_increase_failure  \
Metric          Vessel                                                      
Weather Delay   ROV Support Vessel                                 42.845   
                Cable Laying Vessel                                41.449   
                Anchor Handling Vessel                              0.252   
                Heavy Lift Vessel  

In [9]:
count_means

layout_1_in_situ_base  \
part_name                 reason                                                               
cable                     cable & scour survey                                       610.000   
                          cable replacement                                           16.800   
central_hydraulics_system medium repair                                              422.600   
                          minor repair                                               418.400   
drive_train_system        major repair                                                 1.333   
                          medium repair                                              286.200   
                          minor repair                                               301.200   
electrical_system         major repair                                                 3.500   
                          major replacement                                            2.600   
gearbox                   major repair                                                 6.600   
                          major replacement                                            6.200   
generator_system          major repair                                                 3.800   
                          major replacement                                            2.400   
                          medium repair                                              425.200   
                          minor repair                                               432.000   
main_bearing              major replacement                                            1.000   
mooring_lines             mooring and cable inspection                               580.800   
                          mooring replacement                                          2.600   
rotor_blades              major repair                                                19.000   
                          major replacement                                           20.600   
rotor_system              medium repair                                              674.600   
                          minor repair                                               662.200   
substructure              medium repair                                               71.400   
                          minor repair                                               372.600   
transmission_system       medium repair                                            1,353.200   
                          minor repair                                             1,338.400   
turbine                   annual service - HV components                             720.000   
                          annual service - lifting & safety equipment                720.000   
                          annual service - turbine                                   720.200   
                          medium repair                                              203.600   
                          minor repair                                               196.800   
yaw_system                medium repair                                              881.000   
                          minor repair                                               892.600   

                                                                       layout_1_in_situ_no_major  \
part_name                 reason                                                                   
cable                     cable & scour survey                                           603.800   
                          cable replacement                                                0.000   
central_hydraulics_system medium repair                                                  430.600   
                          minor repair                                                   430.600   
drive_train_system        major repair                                                     0.000   
          

In [10]:
sum_means

layout_1_in_situ_base  \
part_name                 reason                                                               
cable                     cable & scour survey                                 6,100,000.000   
                          cable replacement                                    3,360,000.000   
central_hydraulics_system medium repair                                       32,836,020.000   
                          minor repair                                         1,757,280.000   
drive_train_system        major repair                                            80,000.000   
                          medium repair                                       22,237,740.000   
                          minor repair                                         1,265,040.000   
electrical_system         major repair                                           350,000.000   
                          major replacement                                    1,300,000.000   
gearbox                   major repair                                         1,386,000.000   
                          major replacement                                    6,510,000.000   
generator_system          major repair                                         1,140,000.000   
                          major replacement                                    3,600,000.000   
                          medium repair                                       33,038,040.000   
                          minor repair                                         1,814,400.000   
main_bearing              major replacement                                      300,000.000   
mooring_lines             mooring and cable inspection                         5,808,000.000   
                          mooring replacement                                  1,482,000.000   
rotor_blades              major repair                                         2,470,000.000   
                          major replacement                                   13,390,000.000   
rotor_system              medium repair                                       52,416,420.000   
                          minor repair                                         2,781,240.000   
substructure              medium repair                                        5,547,780.000   
                          minor repair                                         1,564,920.000   
transmission_system       medium repair                                      105,143,640.000   
                          minor repair                                         5,621,280.000   
turbine                   annual service - HV components                       3,600,000.000   
                          annual service - lifting & safety equipment          3,600,000.000   
                          annual service - turbine                             3,601,000.000   
                          medium repair                                       15,819,720.000   
                          minor repair                                           826,560.000   
yaw_system                medium repair                                       68,453,700.000   
                          minor repair                                         3,748,920.000   

                                                                       layout_1_in_situ_no_major  \
part_name                 reason                                                                   
cable                     cable & scour survey                                     6,038,000.000   
                          cable replacement                                                0.000   
central_hydraulics_system medium repair                                           33,457,620.000   
                          minor repair                                             1,808,520.000   
drive_train_system        major repair                                                     0.000   
          